# MODIS snow metrics

MODIS snow metrics are downloaded as multi-band rasters, each band is a
metric and rasters are for years: 2001-2019.

Steps:
1. Create new raster by year filtered on band 10 cells equal to 32.
2. Use zonal statistics to calculate average lcld by watershed.


In [ ]:
# 1.1 create dictionary with path and year for all modis rasters

import arcpy
import os

modis_folder = r"W:\GIS\MODIS_Snow_Metrics"
modis_rasters = {}

arcpy.env.workspace = modis_folder
rasters = arcpy.ListRasters()

#remove extra 2011 raster. not sure what it is.
rasters = [x for x in rasters if "_c.tif" not in x]
rasters = [x for x in rasters if "byyear.tif" not in x]
print(rasters)

for raster in rasters:
    # path = os.path.join(modis_folder, raster)
    year = raster[0:4]
    modis_rasters[year] = path

print(len(modis_rasters))
print(modis_rasters)

In [ ]:
# 1.2 create new set of rasters after filtering on band 10 value
# for land and continuous snow cover

import arcpy, os
from arcpy.sa import *

arcpy.env.overwriteOutput = True

for key, value in modis_rasters.items():
    print("Creating band_5 mask for: " + key)
    #create mask by converting band_10 to 1/0
    band_10 = Raster(value + "/Band_10")
    mask1 = Con(band_10 == 32, 1)
    #extract band_5 to new raster using new mask
    extract1 = ExtractByMask(value + "/Band_5", mask1)
    extract1.save("W://GIS//MODIS_Snow_Metrics//LCLD_rasters//" + key + "_lcld_32.tif")

In [6]:
# 1.3 create dictionary with path and year for all lcld rasters

import arcpy
import os

lcld_folder = r"W:\GIS\MODIS_Snow_Metrics\LCLD_rasters"
lcld_rasters = {}

arcpy.env.workspace = lcld_folder
rasters = arcpy.ListRasters()

for raster in rasters:
    path = os.path.join(lcld_folder, raster)
    year = raster[0:4]
    lcld_rasters[year] = path

print(len(lcld_rasters))
print(lcld_rasters)


19
{'2001': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2001_lcld_32.tif', '2002': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2002_lcld_32.tif', '2003': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2003_lcld_32.tif', '2004': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2004_lcld_32.tif', '2005': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2005_lcld_32.tif', '2006': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2006_lcld_32.tif', '2007': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2007_lcld_32.tif', '2008': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2008_lcld_32.tif', '2009': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2009_lcld_32.tif', '2010': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2010_lcld_32.tif', '2011': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2011_lcld_32.tif', '2012': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2012_lcld_32.tif', '2013': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2013_lcld_32.tif', '2014': 'W:\\GIS\\MODIS_Snow_Metrics\\LCLD_rasters\\2014_lcld_32.tif', '2

Step 2 get average lcld by watershed using zonal statistics as table
Note there is an error for very small watersheds that don't intersect cell center
Used try except to skip error, buffer by 150 meters and try again
MODIS cell size is 500 m so that should work
Adding fields for watershed number and year of MODIS snow metric
appending mean, watershed, and year to a table
deleting zonal stats table
converting everything to pandas data frame and exporting as .csv

In [1]:
import arcpy, os, re
import numpy as np
import pandas as pd
from arcpy.sa import *
arcpy.env.overwriteOutput = True

baseFolder = r"W:\GIS\AKSSF"
# regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]
regions = ["Cook_Inlet", "Copper_River"]

for region in regions:
    print("Starting zonal statistics for region: " + region)
    modis_lst = []
    wtd_dataset = baseFolder + "\\" + region + "\\" + region + ".gdb\\Watersheds"
    arcpy.env.workspace = wtd_dataset
    wtds = arcpy.ListFeatureClasses()
    print(wtds)
    print(len(wtds))
    for wtd in wtds:
        print("starting zonal statistics for wtd: " + wtd)
        wtd_path = os.path.join(wtd_dataset, wtd)
        for key, value in lcld_rasters.items():
            try:
                gdb = baseFolder + "\\" + region + "\\" + region + ".gdb"
                out_table = gdb + "\\" + wtd + "_" + key + "_" + "tbl"
                arcpy.ia.ZonalStatisticsAsTable(wtd_path, "OBJECTID", value, out_table, "DATA", "MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")
            except:
                buffer = wtd + "buf"
                arcpy.Buffer_analysis(wtd_path, buffer, "150 Meters")
                arcpy.ia.ZonalStatisticsAsTable(buffer, "OBJECTID", value, out_table, "DATA", "MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")
            #add fields
            arcpy.AddField_management(out_table, "year", "SHORT")
            arcpy.AddField_management(out_table, "cat_ID", "DOUBLE")
            table_name = os.path.basename(out_table)
            print(table_name)
            m = re.findall('_(.*)_', table_name, re.MULTILINE)
            print(m)
            for a in m:
                cat_id = re.findall(r'[0-9]+', a)[0] #numbers only
                year = re.split("\_", a)[1]
                print(cat_id)
                print(year)
            arcpy.CalculateField_management(out_table, "year", year)
            arcpy.CalculateField_management(out_table, "cat_id", cat_id)
            #convert table to numpy array, append to list
            arr = arcpy.da.TableToNumPyArray(out_table, ['year', 'cat_id', 'MEAN'])
            modis_lst.append(arr)
            print(modis_lst)
            #delete table
            arcpy.Delete_management(out_table)

    #convert list to pandas data frame
    df = pd.DataFrame(np.concatenate(modis_lst))
    df.insert(0, 'Region', region)
    df.to_csv(baseFolder + "\\" + region + "\\modis_summary_table.csv")


Starting zonal statistics for region: Cook_Inlet
['wtd_75004200007057', 'wtd_75004300006312', 'wtd_75004300001906', 'wtd_75004300000100', 'wtd_75004300004983', 'wtd_75004300004332', 'wtd_75004300006239', 'wtd_75004300004304', 'wtd_75004300002332', 'wtd_75004300005437', 'wtd_75004300003464', 'wtd_75004300005171', 'wtd_75004300004701', 'wtd_75004300005303', 'wtd_75004300006440', 'wtd_75004300004254', 'wtd_75004300001207', 'wtd_75004300005245', 'wtd_75004300000311', 'wtd_75004300008012', 'wtd_75004300004192', 'wtd_75004300007324', 'wtd_75004300001162', 'wtd_75004300004217', 'wtd_75004300002207', 'wtd_75004300003409', 'wtd_75004300002142', 'wtd_75004300002314', 'wtd_75004300003171', 'wtd_75004300000101', 'wtd_75004300005352', 'wtd_75004300000856', 'wtd_75004300004341', 'wtd_75004300002289', 'wtd_75004300007331', 'wtd_75004300006376', 'wtd_75004300002278', 'wtd_75004300002352', 'wtd_75004300001210', 'wtd_75004300004452', 'wtd_75004300000105', 'wtd_75004300006071', 'wtd_75004300004324', 'wtd